# Parte 1: Dados - Users_Followers

In [4]:
import pandas as pd
import re
import numpy as np

def fs_to_dataframe(fs):
    return pd.DataFrame({'User': re.split('\W+', fs)})

def create_edges_df(batches):
    return pd.concat(batches,
        ignore_index=True)

In [6]:
# load dataset

gh_users_allFollowers = pd.read_csv('temp_users_all-followers.csv')
gh_users_allFollowers.head()

,User,All_Followers,nF
0,tarruda,"Sannis,danielmahon,csjaba,FergusRedican,Victor...",570
1,mairatma,"brunocoelho,henvic,eduardolundgren,aperrelli,a...",363
2,joselitojunior1,"renatooliveira,jeffesonmaia,jotaefe,duartefq,J...",350
3,marcelcaraciolo,"thiagoarrais,brunojm,henriquebastos,macndesign...",330
4,luanfonceca,"brunohenrique,luizvarela,gladson,lucasbibiano,...",301


In [7]:
# normalizing
gh_users_allFollowers['All_Followers'] = gh_users_allFollowers['All_Followers'].fillna('')
gh_users_allFollowers['All_Followers'] = gh_users_allFollowers['All_Followers'].apply(fs_to_dataframe)

gh_users_allFollowers.head()

,User,All_Followers,nF
0,tarruda,User 0 S...,570
1,mairatma,User 0 brunocoelho ...,363
2,joselitojunior1,User 0 renatooliveira 1 ...,350
3,marcelcaraciolo,User 0 thiagoarrais 1 ...,330
4,luanfonceca,User 0 brunohenrique 1 ...,301


In [21]:
# Dataframe apenas com os seguidores recifenses

gh_users_Followers = gh_users_allFollowers.copy()

gh_users_Followers['Followers'] = gh_users_Followers['All_Followers'].apply(pd.DataFrame({'User':gh_users_Followers['User']}).merge)
gh_users_Followers['nFs'] = gh_users_Followers['Followers'].apply(len)

gh_users_Followers.drop(['All_Followers', 'nF'], axis=1, inplace=True)
gh_users_Followers.sort_values('nFs', axis=0, ascending=False, inplace=True)

gh_users_Followers.head()

,User,Followers,nFs
12,fernandocastor,User 0 joselitojunior1 1 ...,48
8,filipeximenes,User 0 luanfonceca ...,40
7,simoneas02,User 0 talitaolive...,37
9,renatooliveira,User 0 joselitojunior1 1 ...,35
26,lailsonbm,User 0 luanfonceca 1 ...,29


In [22]:
# todos os usuarios recifenses (nem todos serao considerados como nos do graph - ver Parte 2) 
nodes_preview = gh_users_Followers.loc[:,['User','nFs']]
nodes_preview.columns = ['Id', 'In-Degree']

nodes_preview.head()

,Id,In-Degree
12,fernandocastor,48
8,filipeximenes,40
7,simoneas02,37
9,renatooliveira,35
26,lailsonbm,29


In [23]:
# criacao das arestas

pairs = zip(gh_users_Followers.User, gh_users_Followers.Followers)

edges = [pd.DataFrame([{'Source':str(f), 'Target':str(u)} for f in fs['User']]) for (u,fs) in pairs]

edges_df = create_edges_df(edges)
edges_df.head()

,Source,Target
0,joselitojunior1,fernandocastor
1,filipeximenes,fernandocastor
2,renatooliveira,fernandocastor
3,henvic,fernandocastor
4,fjsj,fernandocastor


In [24]:
edges_df.to_csv('edges.csv', index=False)


# Parte 2: Análise dos Dados - Users_Followers

In [25]:
# numero de usuarios recifenses 
gh_users_Followers['User'].count()

1992

In [26]:
# numero de usuarios recifenses com pelo menos 1 seguidor 
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nFs'] != 0]]
len(users_0_fs)

692

In [27]:
# numero de usuarios recifenses com 0 seguidores 
users_0_fs = [str(n) for n in gh_users_Followers.User[gh_users_Followers['nFs'] == 0]]
len(users_0_fs)

1300

Obs.: número de nós considerados

Note que como estes nós pertencem a uma aresta, eles representam os usuários que tem seguidores e/ou seguem alguém.

In [28]:
# (usuarios recifenses, excluidos os que não tem seguidores se e somente se não seguem ninguem)
len(pd.unique(edges_df[['Source','Target']].values.ravel()))

828

In [29]:
# numero de arestas (somatorio do total de seguidores por usuario)
gh_users_Followers.nFs.sum()

2320

In [30]:
# revendo o numero de arestas
edges = pd.read_csv('edges.csv')
edges.count()

Source    2320
Target    2320
dtype: int64

In [31]:
# distribuicao do Grau
gh_users_Followers.nFs.value_counts()

0     1300
1      310
2      142
3       67
4       42
5       27
7       19
6       18
8       14
9       14
11       6
14       5
12       3
23       3
20       2
13       2
18       2
10       2
15       2
24       2
25       2
29       1
37       1
35       1
16       1
28       1
19       1
40       1
48       1
Name: nFs, dtype: int64

In [32]:
# distribuicao do grau - usuarios por ocorrencias da quantidade de seguidores 
groupby_nfs = gh_users_Followers.groupby('nFs')
groupby_nfs['User'].apply( ','.join)

nFs
0     eliene-mb,anniewalker,lucasvsr,nataliaalves,th...
1     filipe-torres,ericksantana,Kassio-Ferreira,koo...
2     sudorafa,gilesv,andreneto,marcoshmendes,tonnyv...
3     luhanlacerda,walber,mauLeal,tcostam,bebetocf,v...
4     vmms16,wwcoderecife,andreldm,lhew,brunovpl,cit...
5     favasconcelos,alinedoleron,djalmaafilho,Master...
6     vanessa,gcaraciolo,luizlago,omailson,rvlb-19,L...
7     pedroqueiroz,t0rr3sp3dr0,leticiamachado,felipe...
8     iagobelo,dvro,nielsonsantana,tomersimis,JoaoGF...
9     patrickrbc,luanfonceca,jeftarmascarenhas,ovict...
10                              miguelarauj1o,Cisneiros
11    lmarinho,leopoldomt,karlafalcao,irgmedeiros,br...
12           brunnogomes,jordanamorais,marcellustavares
13                            brunofarache,alexpessoajr
14    vinicius3w,thiagodiniz,diegonvs,victorlaerte,a...
15                                interaminense,dakerfp
16                                           pauloborba
18                                        gv